In [1]:
# Load products and productcategories from the bronze layer
products_df = spark.read.table("products_bronze")
categories_df = spark.read.table("product_categories_bronze")

StatementMeta(, fde2cc50-ea4c-4fd8-b529-f6d86b6660c7, 3, Finished, Available, Finished)

In [2]:
# Identify products with no matching category or null product category values
orphaned_products_df = products_df.join(
    categories_df,
    on="ProductCategoryID",
    how="left_anti"
)

StatementMeta(, fde2cc50-ea4c-4fd8-b529-f6d86b6660c7, 4, Finished, Available, Finished)

In [3]:
orphaned_products_df.show()

StatementMeta(, fde2cc50-ea4c-4fd8-b529-f6d86b6660c7, 5, Finished, Available, Finished)

+-----------------+---------+------------------+-------+
|ProductCategoryID|ProductID|       ProductName|  Price|
+-----------------+---------+------------------+-------+
|                1|        1|         iPhone 14| 999.99|
|                1|        2|Samsung Galaxy S23| 849.99|
|                1|        3|       MacBook Pro|1999.99|
|                1|        4|       Dell XPS 13|1499.99|
|                2|        5|     Men's T-Shirt|  19.99|
|                2|        6|       Men's Jeans|  39.99|
|                2|        7|     Women's Dress|  59.99|
|                2|        8|    Women's Blouse|  29.99|
|                3|        9|           Blender|  79.99|
|                3|       10|    Microwave Oven| 129.99|
+-----------------+---------+------------------+-------+



In [5]:
# Save to quarantine table in the silver layer
orphaned_products_df.write.mode("overwrite").saveAsTable("Products_Quarantine_silver")

StatementMeta(, c4853f57-d748-4b77-897f-49106a02bcbe, 6, Finished, Available, Finished)

In [6]:
# Join products with categories (filter and enrich in one step)
dim_product_df = products_df.join(
    categories_df,
    on="ProductCategoryID",
    how="inner"
)

StatementMeta(, c4853f57-d748-4b77-897f-49106a02bcbe, 7, Finished, Available, Finished)

In [7]:
dim_product_df.show()

StatementMeta(, c4853f57-d748-4b77-897f-49106a02bcbe, 8, Finished, Available, Finished)

+-----------------+---------+------------------+-------+---------------+
|ProductCategoryID|ProductID|       ProductName|  Price|   CategoryName|
+-----------------+---------+------------------+-------+---------------+
|                1|        1|         iPhone 14| 999.99|    Electronics|
|                1|        2|Samsung Galaxy S23| 849.99|    Electronics|
|                1|        3|       MacBook Pro|1999.99|    Electronics|
|                1|        4|       Dell XPS 13|1499.99|    Electronics|
|                2|        5|     Men's T-Shirt|  19.99|       Clothing|
|                2|        6|       Men's Jeans|  39.99|       Clothing|
|                2|        7|     Women's Dress|  59.99|       Clothing|
|                2|        8|    Women's Blouse|  29.99|       Clothing|
|                3|        9|           Blender|  79.99|Home Appliances|
|                3|       10|    Microwave Oven| 129.99|Home Appliances|
+-----------------+---------+------------------+---

In [8]:
# Save to gold product dimension table
dim_product_df.write.mode("overwrite").saveAsTable("DimProduct_gold")

StatementMeta(, c4853f57-d748-4b77-897f-49106a02bcbe, 9, Finished, Available, Finished)

In [10]:
crossjoin_df = products_df.crossJoin(categories_df)

StatementMeta(, c4853f57-d748-4b77-897f-49106a02bcbe, 11, Finished, Available, Finished)

In [14]:
crossjoin_df.show()

StatementMeta(, c4853f57-d748-4b77-897f-49106a02bcbe, 15, Finished, Available, Finished)

+---------+------------------+-----------------+-------+-----------------+---------------+
|ProductID|       ProductName|ProductCategoryID|  Price|ProductCategoryID|   CategoryName|
+---------+------------------+-----------------+-------+-----------------+---------------+
|        1|         iPhone 14|                1| 999.99|                1|    Electronics|
|        1|         iPhone 14|                1| 999.99|                2|       Clothing|
|        1|         iPhone 14|                1| 999.99|                3|Home Appliances|
|        2|Samsung Galaxy S23|                1| 849.99|                1|    Electronics|
|        2|Samsung Galaxy S23|                1| 849.99|                2|       Clothing|
|        2|Samsung Galaxy S23|                1| 849.99|                3|Home Appliances|
|        3|       MacBook Pro|                1|1999.99|                1|    Electronics|
|        3|       MacBook Pro|                1|1999.99|                2|       Clothing|